In [ ]:
import os
import yaml
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import List, TypedDict, Annotated
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
"""
from langchain.vectorstores import FAISS
from langchain_core.graph import Graph, Node, StartNode, StopNode
from langchain.docstore.document import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
"""

# Load environment variables
load_dotenv()
api_key = os.environ["groq"]

# Initialize LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key
)


In [32]:
llm.invoke("hi").content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [33]:
llm.invoke("where is taj mahal?").content

'The Taj Mahal is located in Agra, a city in the Indian state of Uttar Pradesh. It is situated on the southern bank of the Yamuna River, approximately 230 kilometers (143 miles) south of New Delhi, the capital city of India.\n\nThe exact address of the Taj Mahal is:\n\nTaj Mahal, Dharmapuri, Forest Colony, Tajganj, Agra, Uttar Pradesh 282001, India\n\nIt is one of the most famous and iconic monuments in the world, and a popular tourist destination, attracting millions of visitors each year.'

In [ ]:
resume=""
with open('data/job_description.txt', 'r') as file:
    job_description=file.read()
loader = PyPDFLoader("data/resume_with_capg.pdf")
documents = loader.load()
for i in documents:
    resume+=(i.page_content.strip())
    
with open('prompts.yaml', 'r') as file:
    data = yaml.safe_load(file)
    
def intro():

    print("*"*100)
    for i,j in enumerate(data.keys()):
        print(f"{i+1}. {j}")
def inputs():
    i=input("Enter the number corresponding to the key you want to use: ")
    print("selected number is ",int(i))
    key=list(data.keys())[int(i)-1]
    return key

def func(key):
    input_variables=list(data[key]["inputs"].keys())
    template=data[key]["prompt"]
    prompt_template = PromptTemplate(
        input_variables=input_variables,
        template=template
    )
    print(prompt_template,input_variables)
    chain=prompt_template | llm
    response=chain.invoke(data[key]["inputs"]).content
    return response

In [90]:
intro()


****************************************************************************************************
1. resume_and_job_decription_rewrite
2. cold_mail_and_person_info
3. cold_mail_no_personal_info


In [91]:
x=inputs()

selected number is  1


In [92]:
func(x)

input_variables=['job_description', 'resume'] input_types={} partial_variables={} template='"""\nRewrite the following resume to match the job description provided:\n\nResume: {resume}\n\nJob Description: {job_description}\n\nInstructions:\n1) Use the available experience, projects, and skills to rewrite the resume.\n2) Don\'t Halucinate.\n3) Don\'t use any external data but only the resume and job description.\n4) Don\'t mention any comments and preamble in the response.\n"""\n' ['resume', 'job_description']


"I don't see a resume or job description provided. Please provide the necessary information so I can assist you with rewriting the resume to match the job description."

NameError: name 'a' is not defined

In [61]:
"""
creating a tool for llm that takes user input and uses the func function to get the response


"""

@tool
def get_response(resume: str, job_description: str) -> str: 
    """
    Use this tool to get a response based on the resume and job description.
    """
    #
    response = func(key)
    return response


In [62]:
"""Binding the tool to the LLM"""   
 
tools = [
    Tool(
        name="get_response",
        func=get_response,
        description="Use this tool to get a response based on the resume and job description."
    )
]
llm_with_tools = llm.bind_tools(tools)

In [64]:
llm_with_tools.invoke({"key":"1"})

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.